In [29]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import joblib
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint



pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

In [10]:
df= pd.read_csv(r"C:\Users\anass\Cellula\first inten project.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [11]:
df["date of reservation"] = pd.to_datetime(df["date of reservation"], errors='coerce')
df["date of reservation"] = pd.to_datetime(df["date of reservation"].fillna("02/2/2018").dt.strftime("%m/%d/%Y"))
df['Month'] = df['date of reservation'].dt.month.astype('Int64')
df['Year'] = df['date of reservation'].dt.year.astype('Int64')
df['Day_of_week'] = df['date of reservation'].dt.dayofweek
df['Days of reservation']=(df['number of weekend nights'] + df['number of week nights'])

In [ ]:
df.head()

In [12]:
df.drop(columns=['date of reservation'],inplace=True)

In [ ]:
df.head()

In [13]:
df['Total_Family_Number']=df['number of adults']+df['number of children']
df['Total_Family_Number'] = df['Total_Family_Number'].astype(int)
df.drop(columns=['number of adults','number of children'],axis=1,inplace=True)

In [ ]:
df.head()

In [14]:
def remove_outliers(df, threshold=3):
    cols = ['lead time', 'average price ']
    df_clean = df.copy()  
            # initialize a new dataframe to avoid modifying the original
    for col in cols:
        zscore = (df_clean[col] - df_clean[col].mean()) / df_clean[col].std()
        df_clean = df_clean[abs(zscore) <= threshold]
    return df_clean
df=remove_outliers(df)
df

Booking_ID  number of weekend nights  number of week nights  \
0       INN00001                         2                      5   
1       INN00002                         1                      3   
2       INN00003                         1                      3   
3       INN00004                         0                      2   
4       INN00005                         1                      2   
...          ...                       ...                    ...   
36279   INN36281                         1                      1   
36281   INN36283                         1                      3   
36282   INN36284                         1                      3   
36283   INN36285                         0                      4   
36284   INN36286                         0                      5   

       type of meal  car parking space    room type  lead time  \
0       Meal Plan 1                  0  Room_Type 1        224   
1      Not Selected                  0  Room_Type 1          5   
2       Meal Plan 1                  0  Room_Type 1          1   
3       Meal Plan 1                  0  Room_Type 1        211   
4      Not Selected                  0  Room_Type 1         48   
...             ...                ...          ...        ...   
36279  Not Selected                  0  Room_Type 1         48   
36281   Meal Plan 1                  0  Room_Type 1         34   
36282   Meal Plan 1                  0  Room_Type 4         83   
36283   Meal Plan 1                  0  Room_Type 1        121   
36284   Meal Plan 1                  0  Room_Type 4         44   

      market segment type  repeated  P-C  P-not-C  average price   \
0                 Offline         0    0        0           88.00   
1                  Online         0    0        0          106.68   
2                  Online         0    0        0           50.00   
3                  Online         0    0        0          100.00   
4                  Online         0    0        0           77.00   
...                   ...       ...  ...      ...             ...   
36279              Online         0    0        0           94.50   
36281              Online         0    0        0          107.55   
36282              Online         0    0        0          105.61   
36283             Offline         0    0        0           96.90   
36284              Online         0    0        0          133.44   

       special requests booking status  Month  Year  Day_of_week  \
0                     0   Not_Canceled     10  2015            4   
1                     1   Not_Canceled     11  2018            1   
2                     0       Canceled      2  2018            2   
3                     1       Canceled      5  2017            5   
4                     0       Canceled      4  2018            2   
...                 ...            ...    ...   ...          ...   
36279                 0       Canceled      4  2018            2   
36281                 1   Not_Canceled     10  2017            6   
36282                 1   Not_Canceled     12  2018            2   
36283                 1   Not_Canceled      7  2018            4   
36284                 3   Not_Canceled     10  2018            3   

       Days of reservation  Total_Family_Number  
0                        7                    2  
1                        4                    1  
2                        4                    3  
3                        2                    1  
4                        3                    1  
...                    ...                  ...  
36279                    2                    2  
36281                    4                    2  
36282                    4                    2  
36283                    4                    3  
36284                    5                    2  

[35555 rows x 19 columns]

In [15]:
# One hot encoding
df_complete=pd.get_dummies(df[['type of meal','market segment type','room type']],sparse=False,prefix='category').astype(int)

In [12]:
df_complete

category_Meal Plan 1  category_Meal Plan 2  category_Meal Plan 3  \
0                         1                     0                     0   
1                         0                     0                     0   
2                         1                     0                     0   
3                         1                     0                     0   
4                         0                     0                     0   
...                     ...                   ...                   ...   
36279                     0                     0                     0   
36281                     1                     0                     0   
36282                     1                     0                     0   
36283                     1                     0                     0   
36284                     1                     0                     0   

       category_Not Selected  category_Aviation  category_Complementary  \
0                          0                  0                       0   
1                          1                  0                       0   
2                          0                  0                       0   
3                          0                  0                       0   
4                          1                  0                       0   
...                      ...                ...                     ...   
36279                      1                  0                       0   
36281                      0                  0                       0   
36282                      0                  0                       0   
36283                      0                  0                       0   
36284                      0                  0                       0   

       category_Corporate  category_Offline  category_Online  \
0                       0                 1                0   
1                       0                 0                1   
2                       0                 0                1   
3                       0                 0                1   
4                       0                 0                1   
...                   ...               ...              ...   
36279                   0                 0                1   
36281                   0                 0                1   
36282                   0                 0                1   
36283                   0                 1                0   
36284                   0                 0                1   

       category_Room_Type 1  category_Room_Type 2  category_Room_Type 3  \
0                         1                     0                     0   
1                         1                     0                     0   
2                         1                     0                     0   
3                         1                     0                     0   
4                         1                     0                     0   
...                     ...                   ...                   ...   
36279                     1                     0                     0   
36281                     1                     0                     0   
36282                     0                     0                     0   
36283                     1                     0                     0   
36284                     0                     0                     0   

       category_Room_Type 4  category_Room_Type 5  category_Room_Type 6  \
0                         0                     0                     0   
1                         0                     0                     0   
2                         0                     0                     0   
3                         0                     0                     0   
4                         0                     0                     0   
...                     ...                   ...                   ...   
36279     

In [16]:
df['booking status']=df['booking status'].map({'Not_Canceled':0,'Canceled':1})

In [17]:
df.drop(columns=['type of meal','market segment type','room type'],inplace=True)

In [18]:
joindf=df.join(df_complete)

In [ ]:
joindf.head()

In [19]:
joindf.drop(columns=['Booking_ID','number of weekend nights','number of week nights'],inplace=True)

In [23]:
joindf

car parking space  lead time  repeated  P-C  P-not-C  average price   \
0                      0        224         0    0        0           88.00   
1                      0          5         0    0        0          106.68   
2                      0          1         0    0        0           50.00   
3                      0        211         0    0        0          100.00   
4                      0         48         0    0        0           77.00   
...                  ...        ...       ...  ...      ...             ...   
36279                  0         48         0    0        0           94.50   
36281                  0         34         0    0        0          107.55   
36282                  0         83         0    0        0          105.61   
36283                  0        121         0    0        0           96.90   
36284                  0         44         0    0        0          133.44   

       special requests  booking status  Month  Year  Day_of_week  \
0                     0               0     10  2015            4   
1                     1               0     11  2018            1   
2                     0               1      2  2018            2   
3                     1               1      5  2017            5   
4                     0               1      4  2018            2   
...                 ...             ...    ...   ...          ...   
36279                 0               1      4  2018            2   
36281                 1               0     10  2017            6   
36282                 1               0     12  2018            2   
36283                 1               0      7  2018            4   
36284                 3               0     10  2018            3   

       Days of reservation  Total_Family_Number  category_Meal Plan 1  \
0                        7                    2                     1   
1                        4                    1                     0   
2                        4                    3                     1   
3                        2                    1                     1   
4                        3                    1                     0   
...                    ...                  ...                   ...   
36279                    2                    2                     0   
36281                    4                    2                     1   
36282                    4                    2                     1   
36283                    4                    3                     1   
36284                    5                    2                     1   

       category_Meal Plan 2  category_Meal Plan 3  category_Not Selected  \
0                         0                     0                      0   
1                         0                     0                      1   
2                         0                     0                      0   
3                         0                     0                      0   
4                         0                     0                      1   
...                     ...                   ...                    ...   
36279                     0                     0                      1   
36281                     0                     0                      0   
36282                     0                     0                      0   
36283                     0                     0                      0   
36284                     0                     0                      0   

       category_Aviation  category_Complementary  category_Corporate  \
0                      0                       0                   0   
1                      0                       0                   0   
2                      0                       0                   0   
3                      0                       0                   0   
4                      0                       0                   0   
...    

In [20]:
X = joindf.drop('booking status', axis=1)
y = joindf['booking status']

In [27]:

lasso = LassoCV(cv=5, random_state=42)
lasso.fit(X, y)

# Get selected feature names
selected_features = X.columns[lasso.coef_ != 0]
print(selected_features)
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE

# Example with Random Forest Classifier
estimator = RandomForestClassifier(n_estimators=100, random_state=42)
selector = RFE(estimator, n_features_to_select=5, step=1)
selector.fit(X, y)

# Get selected feature names
selected_features = X.columns[selector.support_]
print(selected_features)
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# Example with Logistic Regression
estimator = LogisticRegression()
selector = RFE(estimator, n_features_to_select=5, step=1)
selector.fit(X, y)

# Get selected feature names
selected_features = X.columns[selector.support_]
print(selected_features)

Index(['lead time', 'average price ', 'special requests', 'Month',
       'Days of reservation', 'category_Offline', 'category_Online'],
      dtype='object')
Index(['lead time', 'average price ', 'special requests', 'Month',
       'Days of reservation'],
      dtype='object')
Index(['car parking space', 'repeated', 'special requests',
       'category_Complementary', 'category_Online'],
      dtype='object')


In [18]:
#feature Importance
rf = RandomForestClassifier(n_estimators=100, random_state=42)

rf.fit(X, y)

# Extract feature importances
importances = rf.feature_importances_
feature_names = X.columns
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})

# Rank features by importance
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
print(feature_importance_df)

                   Feature  Importance
1                lead time    0.339231
5           average price     0.174322
6         special requests    0.103902
7                    Month    0.093088
10     Days of reservation    0.071529
9              Day_of_week    0.065665
11     Total_Family_Number    0.029096
20         category_Online    0.026590
8                     Year    0.025654
19        category_Offline    0.017012
12    category_Meal Plan 1    0.007026
0        car parking space    0.006721
13    category_Meal Plan 2    0.006627
18      category_Corporate    0.006563
21    category_Room_Type 1    0.006140
24    category_Room_Type 4    0.005776
15   category_Not Selected    0.004984
22    category_Room_Type 2    0.001924
2                 repeated    0.001793
25    category_Room_Type 5    0.001740
4                  P-not-C    0.001406
26    category_Room_Type 6    0.001288
17  category_Complementary    0.000731
16       category_Aviation    0.000433
3                      P-

In [19]:
feature_importance_df['Feature'].head(10)

1               lead time
5          average price 
6        special requests
7                   Month
10    Days of reservation
9             Day_of_week
11    Total_Family_Number
20        category_Online
8                    Year
19       category_Offline
Name: Feature, dtype: object

In [20]:
feature_importance_df['Feature'].values

array(['lead time', 'average price ', 'special requests', 'Month',
       'Days of reservation', 'Day_of_week', 'Total_Family_Number',
       'category_Online', 'Year', 'category_Offline',
       'category_Meal Plan 1', 'car parking space',
       'category_Meal Plan 2', 'category_Corporate',
       'category_Room_Type 1', 'category_Room_Type 4',
       'category_Not Selected', 'category_Room_Type 2', 'repeated',
       'category_Room_Type 5', 'P-not-C', 'category_Room_Type 6',
       'category_Complementary', 'category_Aviation', 'P-C',
       'category_Room_Type 7', 'category_Meal Plan 3',
       'category_Room_Type 3'], dtype=object)

In [21]:
final_df=joindf[['lead time', 'average price ', 'special requests', 'Month',
       'Days of reservation', 'Day_of_week', 'Total_Family_Number',
       'category_Online', 'Year', 'category_Offline']]

In [22]:
final_df

lead time  average price   special requests  Month  \
0            224           88.00                 0     10   
1              5          106.68                 1     11   
2              1           50.00                 0      2   
3            211          100.00                 1      5   
4             48           77.00                 0      4   
...          ...             ...               ...    ...   
36279         48           94.50                 0      4   
36281         34          107.55                 1     10   
36282         83          105.61                 1     12   
36283        121           96.90                 1      7   
36284         44          133.44                 3     10   

       Days of reservation  Day_of_week  Total_Family_Number  category_Online  \
0                        7            4                    2                0   
1                        4            1                    1                1   
2                        4            2                    3                1   
3                        2            5                    1                1   
4                        3            2                    1                1   
...                    ...          ...                  ...              ...   
36279                    2            2                    2                1   
36281                    4            6                    2                1   
36282                    4            2                    2                1   
36283                    4            4                    3                0   
36284                    5            3                    2                1   

       Year  category_Offline  
0      2015                 1  
1      2018                 0  
2      2018                 0  
3      2017                 0  
4      2018                 0  
...     ...               ...  
36279  2018                 0  
36281  2017                 0  
36282  2018                 0  
36283  2018                 1  
36284  2018                 0  

[35555 rows x 10 columns]

In [22]:
final_df.to_csv('data.csv', index= False)

In [31]:
X_train, X_test, y_train, y_test = train_test_split(final_df, y, test_size=0.2, random_state=40)

In [32]:
# Initialize Logistic Regression model
model = LogisticRegression()

# Fit the model
model.fit(X_train, y_train)

# Predictions on the test set
y_pred = model.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Confusion matrix
confusion_mat = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(confusion_mat)

Accuracy: 0.7984812262691604
Confusion Matrix:
[[4327  508]
 [ 925 1351]]


### Hyperparameter Tuning

In [ ]:

n_estimators = [100, 500]
max_features = ['sqrt', 'log2']

grid = dict(n_estimators=n_estimators,max_features=max_features)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=RandomForestClassifier(), param_grid=grid, n_jobs=-1,
                            cv=cv, scoring='accuracy',error_score=0)

# grid_search.fit(X, y)
grid_search.fit(X_train, y_train)

model = grid_search.best_estimator_
best_score = grid_search.best_score_
best_params = grid_search.best_params_


print("Best: %f using %s" % (best_score, best_params))
#0.893920 

In [ ]:
param_dist = {'max_depth': [20],
              'max_features': ['auto'],
              'min_samples_leaf': [1],
              'min_samples_split': [2],
              'n_estimators': [100]}

# Initialize the RandomForestClassifier
rf = RandomForestClassifier(random_state=42)

# Setup RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, 
                                   n_iter=100,  # Number of parameter settings that are sampled
                                   cv=5, scoring='accuracy', verbose=1, n_jobs=-1, random_state=42)

# Perform randomized search to find the best hyperparameters
random_search.fit(X_train, y_train)

# Get the best model
best_rf = random_search.best_estimator_

# Evaluate the best model on validation set
y_pred = best_rf.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy:.4f}')

# Evaluate the best model on test set (assuming X_test, y_test are available)
test_accuracy = accuracy_score(y_test, best_rf.predict(X_test))
print(f'Test Accuracy: {test_accuracy:.4f}')

# You can also access the best hyperparameters found
best_params = random_search.best_params_
print('Best Hyperparameters:', best_params)

In [32]:
rf = RandomForestClassifier()

rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)

accuracy_rf = accuracy_score(y_test, y_pred_rf)

print("Random Forest Accuracy:", accuracy_rf)

# Confusion matrix

confusion_mat_rf = confusion_matrix(y_test, y_pred_rf)

print("Confusion Matrix:")
print(confusion_mat_rf)

Random Forest Accuracy: 0.9009984531008297
Confusion Matrix:
[[4567  268]
 [ 436 1840]]


In [35]:
joblib.dump(rf, 'Random_forest.joblib')

['Random_forest.joblib']

In [36]:
rf = joblib.load('Random_forest.joblib')

In [39]:
y_pred_rf

array([1, 0, 1, ..., 1, 1, 1], dtype=int64)

In [38]:
predictions = rf.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
predictions
accuracy_rf

0.9009984531008297

In [ ]:
dt = DecisionTreeClassifier(random_state=40)

# Fit the model
dt.fit(X_train, y_train)

# Predictions on the test set
y_pred_dt = dt.predict(X_test)

# Evaluate accuracy
accuracy_dt = accuracy_score(y_test, y_pred_dt)
print("Decision Tree Accuracy:", accuracy_dt)

# Confusion matrix
confusion_mat_dt = confusion_matrix(y_test, y_pred_dt)
print("Confusion Matrix:")
print(confusion_mat_dt)